In [4]:
from collections import defaultdict
import os
import pickle
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

In [5]:
def open_report(pickle_file, verbose=True):
    with open(pickle_file, 'br') as f:
        report = pickle.load(f)

    if verbose:
        print('\nfile_name: {}\nmodel:{}'.format(pickle_file, report['model']))
    
    return report

def KLDivergence(P, Q):
    sdklist = []
    for i, (p, q) in enumerate(zip(P, Q)):
        sdk = 0 if p == 0 else p*np.log2(p/q)
        sdk += 0 if p == 1 else (1-p) * np.log2((1-p)/(1-q))
        sdklist.append(sdk)

    return np.array(sdklist)


In [6]:
models_name_groups = [
    ['RMch',
    'RMchD',],

    ['C5o6RMch',
    'C5o6RMchD',
    'C5o6RMchMax',],

    ['C5o6C5o12RMch',
    'C5o6C5o12RMchD',
    'C5o6C5o12RMchMax',],

    ['C5o6C5o12Rfl100Mch',
    'C5o6C5o12Rfl100MchD',
    'C5o6C5o12Rfl100MchMax',],

    ['C5o6C5o12C5o36C5o36RMch',
    'C5o6C5o12C5o36C5o36RMchD',
    'C5o6C5o12C5o36C5o36RMchMax',],

    ['C5o6C5o12C5o36C5o36Rfl100Mch',
    'C5o6C5o12C5o36C5o36Rfl100MchD',
    'C5o6C5o12C5o36C5o36Rfl100MchMax',],
]

models_name = [
    'RMch',
    'RMchD',
    'C5o6RMch',
    'C5o6RMchD',
    'C5o6RMchMax',
    'C5o6C5o12RMch',
    'C5o6C5o12RMchD',
    'C5o6C5o12RMchMax',
    'C5o6C5o12Rfl100Mch',
    'C5o6C5o12Rfl100MchD',
    'C5o6C5o12Rfl100MchMax',
    'C5o6C5o12C5o36C5o36RMch',
    'C5o6C5o12C5o36C5o36RMchD',
    'C5o6C5o12C5o36C5o36RMchMax',
    'C5o6C5o12C5o36C5o36Rfl100Mch',
    'C5o6C5o12C5o36C5o36Rfl100MchD',
    'C5o6C5o12C5o36C5o36Rfl100MchMax',
]

models_alias = {
    'RMch': '$M$',
    'RMchD': '$MD$',
    'C5o6RMch': '$C_6M$',
    'C5o6RMchD': '$C_6MD$',
    'C5o6RMchMax': '$C_6MMax$',    
    'C5o6C5o12RMch': '$C_6C_{12}M$',
    'C5o6C5o12RMchD': '$C_6C_{12}MD$',
    'C5o6C5o12RMchMax': '$C_6C_{12}MMax$',
    'C5o6C5o12Rfl100Mch': '$C_6C_{12}Fl_{100}M$',
    'C5o6C5o12Rfl100MchD': '$C_6C_{12}Fl_{100}MD$',
    'C5o6C5o12Rfl100MchMax': '$C_6C_{12}Fl_{100}MMax$',
    'C5o6C5o12C5o36C5o36RMch': '$C_6C_{12}C_{36}C_{36}M$',
    'C5o6C5o12C5o36C5o36RMchD': '$C_6C_{12}C_{36}C_{36}MD$',
    'C5o6C5o12C5o36C5o36RMchMax': '$C_6C_{12}C_{36}C_{36}MMax$',
    'C5o6C5o12C5o36C5o36Rfl100Mch': '$C_6C_{12}C_{36}C_{36}Fl_{100}M$',
    'C5o6C5o12C5o36C5o36Rfl100MchD': '$C_6C_{12}C_{36}C_{36}Fl_{100}MD$',
    'C5o6C5o12C5o36C5o36Rfl100MchMax': '$C_6C_{12}C_{36}C_{36}Fl_{100}MMax$',
}

assert len(models_name) == len(models_alias)
assert np.all([n in models_alias.keys() for n in models_name])

models_by_name = defaultdict(list)

basepath = './partial_complete/'
pickle_files = [basepath + n for n in  os.listdir(basepath) if '.pkl' in n]

for pickle_file in pickle_files:
    report = open_report(pickle_file, verbose=False)
    model_name = str(report['model_name'])   
    models_by_name[model_name].append(report)
    
for model_name in models_name:
    assert model_name in models_by_name.keys()

for model, reports in models_by_name.items():
    assert len(reports) == 2


In [7]:
def print_model_tabular(model, alias, time_desc):
    total_params = 0
    
    is_maxout = ('Max' in alias)
    
    print('\\begin{table}[!ht]')
    print('{} {}\\newline'.format(alias, time_desc))
    
    #print('\\noindent')
    #print(alias)
    #print('\\newline')
    #print('\\noindent')
    
    print('\\begin{tabularx}{\linewidth}{ |c|X|c|c|c| }')
    print('\hline')
    print('Camada & Descrição & Entrada & Saída & Parâmetros \\\\ \hline')
    for layer in model:
        if 'InputLayer' in layer:
            continue
        
        res = re.search('\(\((.*)\) -> \((.*)\)\)', layer.replace('?, ', ''))
        size_in = [int(x) for x in res.group(1).split(',')]
        size_out = [int(x) for x in res.group(2).split(',')]
        
        layer = re.search('\..*\.(.*?)\(', layer).group(1)
        params = 0
        desc = ''
        if 'ConvLayer' in layer:
            layer_desc = '$C_{{{}}}$'.format(size_out[-1])
            params = 25 *  size_in[-1] * size_out[-1]
            step = 2 if (size_in[0] - 5 + 1)/size_out[0] > 1 else 1
            desc_format = 'Convolucional com {} canais de entrada e {} de saída. Passo da transformação {}.'.format
            desc = desc_format(size_in[-1], size_out[-1], step)
            
            if is_maxout:
                desc += ' Maxout.'
            
            
        elif 'LinearReshapeLayer' in layer:
            layer_desc = 'Lin'
            desc = '\\multicolumn{1}{c|}{-}'
        elif 'LinearMultiCharOutputLayer' in layer:
            layer_desc = '$M$'
            desc = '5 classificadores.'
            params = 180 * size_in[-1]
        elif 'LinearLayer' in layer:
            layer_desc = '$Fl_{{{}}}$'.format(size_out[-1])
            desc = 'Camada densa com {} sinais de entrada e {} sinais de saída.'.format(size_in[-1], size_out[-1])
            params = size_in[-1] * size_out[-1]
        elif 'DropOutLayer' in layer:
            layer_desc = 'Dropout'
            desc = 'Dropout com $p_{drop} = 30\%$.'
            params = 0
            
        total_params += params
        print('{} & {} & ({}) & ({}) & {} \\\\ \hline'.format(layer_desc, desc, ','.join(map(str, size_in)), ','.join(map(str, size_out)), params))
    
    print(' total &  &  &  & {} \\\\ \hline'.format(total_params))
    print('\end{tabularx}')
    print('\end{table}')


def print_model_times(report):
    mean_train_time = report['df_train']['train_time'].as_matrix()[1:].mean()
    mean_total_time = report['df_train']['total_time'].as_matrix()[1:].mean()
    print('\\noindent')
    print('$\\tilde{{\\tau}} = {:.2f}$ segundos, $\\tau = {:.2f}$ segundos.'.format(mean_train_time, mean_total_time))
    print('\\newline')

def print_model_performance(report):
    lrate = report['lrange'][0]
    train_series = report['df_train'].iloc[-1]
    test_series = report['df_test'].iloc[-1]
    
    jtrain, ptrain = train_series[['loss_avg', 'wacc']]
    jtest, ptest = test_series[['loss_avg', 'wacc']]

    rows = []
    rows += [('$J_{{{}}}$'.format(i), 'loss_{}'.format(i)) for i in range(5)]
    rows += [('$p_{{{}}}$'.format(i), 'acc_{}'.format(i)) for i in range(5)]
    rows += [('$J$', 'loss_avg')]
    rows += [('$\hat{{p}}$', 'wacc')]
    
    print()
    print('\\noindent')
    print('\\begin{tabularx}{\linewidth}{|X|X|X|}')
    print('\multicolumn{{3}}{{c}}{{$l_r = {}$}} \\\\ \hline \hline'.format(lrate))
    print('& Treino & Validação \\\\ \hline \hline')
    
    for row in rows:
        tex_name, pandas_name = row
        print('{} & {:.6f} & {:.6f} \\\\ \hline'.format(tex_name, train_series[pandas_name], test_series[pandas_name]))
    
    print('\end{tabularx}')
    
    return


In [8]:
for name_group in models_name_groups:
    model_alias = models_alias[name_group[0]]
    ref_time = models_by_name[name_group[0]][1]['df_train'][['train_time', 'total_time']].sum()
    print('\n\section*{' + model_alias + '[D]' + ('[Max]' if len(name_group) == 3 else '') +'}\n')
    
    for model_name in name_group:
        reports = models_by_name[model_name]
        model_alias = models_alias[model_name]
        report = reports[1]
        
        model_time = report['df_train'][['train_time', 'total_time']].sum()
        model_time_rel = list(100 * (model_time/ref_time - 1).as_matrix())
        
        model_time = list(model_time.as_matrix()) + model_time_rel
        
        time_desc = '$\\tilde{{\\tau}}={0:.2f} (+{2:.2f}\%), \\tau={1:.2f} (+{3:.2f}\%)$'.format(*model_time)
        
        print_model_tabular(report['model'], model_alias, time_desc)
        print()


\section*{$M$[D]}

\begin{table}[!ht]
$M$ $\tilde{\tau}=726.29 (+0.00\%), \tau=1009.74 (+0.00\%)$\newline
\begin{tabularx}{\linewidth}{ |c|X|c|c|c| }
\hline
Camada & Descrição & Entrada & Saída & Parâmetros \\ \hline
Lin & \multicolumn{1}{c|}{-} & (50,200,3) & (30000) & 0 \\ \hline
$M$ & 5 classificadores. & (30000) & (5,36) & 5400000 \\ \hline
 total &  &  &  & 5400000 \\ \hline
\end{tabularx}
\end{table}

\begin{table}[!ht]
$MD$ $\tilde{\tau}=759.56 (+4.58\%), \tau=1081.43 (+7.10\%)$\newline
\begin{tabularx}{\linewidth}{ |c|X|c|c|c| }
\hline
Camada & Descrição & Entrada & Saída & Parâmetros \\ \hline
Lin & \multicolumn{1}{c|}{-} & (50,200,3) & (30000) & 0 \\ \hline
Dropout & Dropout com $p_{drop} = 30\%$. & (30000) & (30000) & 0 \\ \hline
$M$ & 5 classificadores. & (30000) & (5,36) & 5400000 \\ \hline
 total &  &  &  & 5400000 \\ \hline
\end{tabularx}
\end{table}


\section*{$C_6M$[D][Max]}

\begin{table}[!ht]
$C_6M$ $\tilde{\tau}=669.60 (+0.00\%), \tau=912.38 (+0.00\%)$\newline
\be

In [15]:
def build_report_dict(report):
    df_test = report['df_test']
    df_train = report['df_train']
    train_time_series = df_test['train_time'].as_matrix()[1:]
    total_time_series = df_test['total_time'].as_matrix()[1:]
    return {
        'name': report['model_name'],
        'lrate': report['lrange'][0],
        'train_loss': df_train['loss_avg'].iloc[-1],
        'test_loss': df_test['loss_avg'].iloc[-1],

        'train_wacc': df_train['wacc'].iloc[-1],
        'test_wacc': df_test['wacc'].iloc[-1],

        'train_wprob': df_train['wprob'].iloc[-1],
        'test_wprob': df_test['wprob'].iloc[-1],

        'test_wacc_max': df_test['wacc'].max(),
        'mean_time': train_time_series.mean(),
        'std_time': train_time_series.std(),
        'total_std_time': total_time_series.std(),
        'total_mean_time': total_time_series.mean(),
        'size': report['model_size'],
    }

def explore_performance(group):
    model_name = group['name'].iloc[0]
    alias1 = models_alias[model_name]
    
    columns = ['lrate', 'train_loss', 'train_wacc', 'train_wprob', 'test_loss', 'test_wprob', 'test_wacc']
    data = group[columns].sort_values('lrate', ascending=False)
    data['loss_rel'] = data['test_loss']/data['train_loss'] - 1
    data['q_p'] = data['test_wprob']/(data['train_wacc'] + 1e-10)
    
    #return data[['lrate', 'test_loss', 'q_p']]
    
    formatters = {
        'lrate': lambda x: re.sub('1e-0(\d)', '10^{-\g<1>}', '${:1.0e}$'.format(x)),
        'train_loss': lambda x: re.sub('e-0(\d)', '\,10^{-\g<1>}', '${:1.2e}$'.format(x)),
        'train_wacc': lambda x: '${:.2f}$'.format(x),
        'train_wprob': lambda x: '${:.2f}$'.format(x),        
        'test_loss': lambda x: re.sub('e-0(\d)', '\,10^{-\g<1>}', '${:1.2e}$'.format(x)),
        'test_wacc': lambda x: '${:.2f}$'.format(x),
        'loss_rel': lambda x: '${:.2f}$'.format(x),
        'test_wprob': lambda x: '${:.2f}$'.format(x),
    }
    
    for key, form in formatters.items():
        data[key] = data[key].apply(form)
    
    #data = data[['lrate', 'train_loss', 'train_wacc', 'train_wprob', 'test_loss', 'test_wacc', 'train_wprob', 'loss_rel']]
    #line_form = '& {} & {} & {} & {} & {} & {} & {} & {} \\\\ \cline{{2-9}}'.format
    
    data = data[['lrate', 'train_loss', 'train_wacc', 'test_loss', 'test_wacc', 'loss_rel']]
    line_form = '& {} & {} & {} & {} & {} & {} \\\\ \cline{{2-7}}'.format

    
    first_line = '\hline\hline\n\multirow{{2}}{{*}}{{{}}}\n'.format(alias1)    
    
    lines = []
    for row in data.as_matrix():
        lines.append(line_form(*row))    
    
    return first_line + '\n'.join(lines)

for name_group in models_name_groups:
    for model_name in name_group:
        reports = models_by_name[model_name]
        
        selected = []
        for report in reports:
            lrate = report['lrange'][0]
            if lrate in [1e-3, 1e-4]:
                selected.append(build_report_dict(report))
        
        print(explore_performance(pd.DataFrame(selected)))
        


\hline\hline
\multirow{2}{*}{$M$}
& $10^{-3}$ & $1.81\,10^{-1}$ & $0.37$ & $3.98\,10^{-1}$ & $0.21$ & $1.20$ \\ \cline{2-7}
& $10^{-4}$ & $4.77\,10^{-2}$ & $0.53$ & $9.34\,10^{-2}$ & $0.30$ & $0.96$ \\ \cline{2-7}
\hline\hline
\multirow{2}{*}{$MD$}
& $10^{-3}$ & $1.50\,10^{-1}$ & $0.44$ & $3.31\,10^{-1}$ & $0.26$ & $1.20$ \\ \cline{2-7}
& $10^{-4}$ & $7.59\,10^{-2}$ & $0.49$ & $1.17\,10^{-1}$ & $0.35$ & $0.55$ \\ \cline{2-7}
\hline\hline
\multirow{2}{*}{$C_6M$}
& $10^{-3}$ & $2.88\,10^{-3}$ & $0.96$ & $1.20\,10^{-1}$ & $0.49$ & $40.49$ \\ \cline{2-7}
& $10^{-4}$ & $2.41\,10^{-2}$ & $0.74$ & $7.34\,10^{-2}$ & $0.41$ & $2.05$ \\ \cline{2-7}
\hline\hline
\multirow{2}{*}{$C_6MD$}
& $10^{-3}$ & $2.80\,10^{-3}$ & $0.96$ & $6.56\,10^{-2}$ & $0.56$ & $22.40$ \\ \cline{2-7}
& $10^{-4}$ & $3.27\,10^{-2}$ & $0.72$ & $6.02\,10^{-2}$ & $0.51$ & $0.84$ \\ \cline{2-7}
\hline\hline
\multirow{2}{*}{$C_6MMax$}
& $10^{-3}$ & $2.26\,10^{-3}$ & $0.97$ & $1.01\,10^{-1}$ & $0.54$ & $43.56$ \\ \cline{2-7}
& $

In [ ]:
for name_group in models_name_groups:
    for model_name in name_group:
        reports = models_by_name[model_name]
        model_alias = models_alias[model_name]

        report = reports[1]
        size = report['model_size']
        
        print_model_tabular(report['model'], model_alias)
        print()




In [127]:
for model_name in models_name:
    reports = models_by_name[model_name]
    model_alias = models_alias[model_name]
    
    report = reports[1]
    size = report['model_size']
    
    print()
    print('%==================================')
    print('%{}'.format(model_name))
    print('\section*{{{}}}'.format(model_alias))
    
    print_model_times(report)
    print_model_tabular(report['model'], model_alias)
    
    
    for report in sorted(reports, key=lambda x: x['lrange'][0], reverse=True):
        print_model_performance(report)



%==================================
%RMch
\section*{$M$}
\noindent
$\tilde{\tau} = 72.63$ segundos, $\tau = 100.97$ segundos.
\newline
\noindent
\begin{tabularx}{\linewidth}{ |c|X|c|c|c| }
\hline
Camada & Descrição & Entrada & Saída & Parâmetros \\ \hline
Lin & - & (50,200,3) & (30000) & 0 \\ \hline
$M$ & 5 classificadores. & (30000) & (5,36) & 5400000 \\ \hline
 total &  &  &  & 5400000 \\ \hline
\end{tabularx}

\noindent
\begin{tabularx}{\linewidth}{|X|X|X|}
\multicolumn{3}{c}{$l_r = 0.001$} \\ \hline \hline
& Treino & Validação \\ \hline \hline
$J_{0}$ & 0.004722 & 0.011577 \\ \hline
$J_{1}$ & 0.100598 & 0.253641 \\ \hline
$J_{2}$ & 0.199416 & 0.455705 \\ \hline
$J_{3}$ & 0.388860 & 0.762492 \\ \hline
$J_{4}$ & 0.210407 & 0.506661 \\ \hline
$p_{0}$ & 0.994640 & 0.988400 \\ \hline
$p_{1}$ & 0.888880 & 0.794400 \\ \hline
$p_{2}$ & 0.783080 & 0.666000 \\ \hline
$p_{3}$ & 0.640040 & 0.498000 \\ \hline
$p_{4}$ & 0.751560 & 0.597600 \\ \hline
$J$ & 0.180800 & 0.398015 \\ \hline
$\hat{{p}